# Segmenting and Clustering Neighborhoods in Toronto

### 1. Scrape the wikipedia page containing the postal codes of Toronto to obtain the data on boroughs and neighborhoods. Transform the data into pandas dataframe.

For web scraping we'll be using BeautifulSoup package.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Get the page and pass it to BeautifulSoup, along with the parser (we'll use lxml parser).

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page=requests.get(url).text

In [3]:
soup = BeautifulSoup(wiki_page, 'lxml')

Find the table containing the postal codes. There are several tables on this page. The one we need is of class "wikitable sortable".

In [4]:
table = soup.find('table', class_="wikitable sortable")

Get all the rows of the table. The first row is the table header, we need to skip it.

In [5]:
tr = table.find_all('tr')
print(tr[0])

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>


In [6]:
print(tr[1])

<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>


In [7]:
print(len(tr))

289


Create a pandas dataframe. Set PostalCode as index, to facilitate search and insertion.

In [8]:
df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])
df.set_index('PostalCode', inplace=True)
df

,Borough,Neighborhood
PostalCode,,


The following code populates the dataframe. It iterates over the rows and extracts table data. If the borough is not assigned, it skips the row. If the borough is assigned, but the neighborhood is not assigned, it sets the neighborhood to the same value as the borough. If the postal code already exists in the table, we append the new neighborhood value to the existing neighborhood value for this code. We assume that the borough is the same for the same postal code.

In [9]:
#Iterate over the rows, skipping the header
for i in range(1, len(tr)-1):
    row = tr[i]
    td = row.find_all('td')
    code = td[0].text.strip()
    if td[1].text.strip() == 'Not assigned':
        continue
    else:
        borough = td[1].text.strip()
       
    if td[2].text.strip() == 'Not assigned':
        neigh = borough
    else:
        neigh = td[2].text.strip()
    
    #Check if the row with this postal code already exists in the table.
    if code in df.index:
        oldNeigh = df.loc[code, 'Neighborhood']
        neigh = oldNeigh + ", " + neigh
        df.loc[code, 'Neighborhood'] = neigh
    else:
        df.loc[code] = [borough, neigh]

#reset the index, so the table is in the original form.
df = df.reset_index()
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [10]:
df.shape

(103, 3)

## 2. Get coordinates of each neighborhood

#### Get the csv file that contains the coordinates of the neighborhoods:

In [11]:
!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data

--2019-04-25 04:54:50--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.72.228
Connecting to cocl.us (cocl.us)|159.8.72.228|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-04-25 04:54:50--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|159.8.72.228|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-04-25 04:54:51--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-04-25 04:54:51--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mv

#### Read the csv file into pandas dataframe:

In [12]:
df_coord = pd.read_csv("Geospatial_Coordinates.csv")
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df_coord.shape

(103, 3)

#### Rename the column so that the column name is consistent between the two dataframes. Then merge the tables on the PostalCode.

In [14]:
df_coord.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_coord.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df_toronto = pd.merge(df, df_coord, on='PostalCode')
df_toronto.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [16]:
df_toronto.shape

(103, 5)